In [ ]:
from bddl.knowledge_base import *

In [ ]:
# Load particle params and delete everything that shows up there.
import json
particle_size_option = {}
with open(r"D:/ig_pipeline/metadata/diced_particle_systems_colors.json") as f:
    for particle, infos in json.load(f).items():
        particle_size_option[particle] = infos["option"].split(" ")[0]

In [ ]:
import string

raw_prompt = """
You are a helpful assistant that generates particle dimension annotations for particles in a dataset of 3D models.

These particles all correspond to the little pieces that you would obtain by cutting the object into small cubes with a knife, i.e. dicing.
In everyday life, these particles are often used in cooking or as ingredients in various dishes.
They are typically small, uniform pieces that can be easily mixed or cooked with other ingredients.

We will represent each particle as a 3D cube with a single parameter, a size, which is the length of one side of the cube in centimeters.
This needs to be different for each particle type, since in real life we typically don't dice e.g. onions to the same size as steak. We
will provide you with the dictionary definition of the particle system if we have one. We will also provide you with a human annotator's
guess as to whether the particle system is a "fine" one like for onions or a "coarse" one for steak.

Give me the estimated particle size of the below particle system in JSON format. The output should be a JSON
dictionary containing one key "size", whose value is the particle size in centimeters. Output only the JSON - no additional conversation.
"""

valid_chars = set(string.ascii_lowercase) | {" "}
def valid_name(s):
    return True
    return all(c in valid_chars for c in s)

In [ ]:
from getpass import getpass
OPENAI_API_TOKEN = ""   # getpass()

In [ ]:
# USE_OLLAMA, model_id
MODELS = {
  # "gemma3:27b": (True, None),
  # "llama4:scout": (True, None),
  "google/gemini-2.5-flash-preview": (False, None),
  "openai/gpt-4o-mini": (False, None),
  "meta-llama/llama-4-maverick": (False, None), # ["lambda", "deepinfra", "novita"]),
  "qwen/qwen3-235b-a22b": (False, None), # ["deepinfra"]),
}

In [ ]:
import asyncio
from openai import AsyncOpenAI
from ollama import AsyncClient
from asynciolimiter import Limiter

# Limit to 2 requests per second
rate_limiter = Limiter(1)

import json

def strip_code_tags(msg):
    if msg.startswith("```json"):
        msg = msg[8:]
    if msg.startswith("```"):
        msg = msg[3:]
    if msg.endswith("```"):
        msg = msg[:-3]
    return msg.strip()

ollama_client = AsyncClient()
async def query_model_ollama(model_id, obj_name, message, images=None):
    messages = [
        {"role": "system", "content": raw_prompt},
        {"role": "user", "content": message},
    ]
    if images is not None:
        messages[1]["images"] = images
    response = await ollama_client.chat(
        model=model_id,
        messages=messages,
    )
    response_message = response.message.content
    return obj_name, json.loads(strip_code_tags(response_message))

openai_client = AsyncOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENAI_API_TOKEN
)
async def query_model_openai(model_id, obj_name, message, images=None, providers=None):
    await rate_limiter.wait()
    content = [
        {
            "type": "text",
            "text": message,
        }
    ]
    if images is not None:
        for base64_image in images:
            data_url = f"data:image/jpeg;base64,{base64_image}"
            content.append({
                "type": "image_url",
                "image_url": {
                    "url": data_url
                }
            })
    messages = [
        {"role": "system", "content": raw_prompt},
        {"role": "user", "content": content},
    ]
    response = await openai_client.chat.completions.create(
        model=model_id,
        # response_format={
        #     "type": "json_schema",
        #     "json_schema": {
        #         "name": "object_mass",
        #         "strict": True,
        #         "schema": {
        #             "type": "object",
        #             "properties": {
        #                 "mass": {
        #                     "type": "number",
        #                     "description": "Mass of object category in kilograms"
        #                 },
        #             },
        #             "required": ["mass"],
        #             "additionalProperties": False
        #         }
        #     }
        # },
        messages=messages,
        extra_body={"provider": {"only": providers}} if providers is not None else None,
    )
    # print(response)
    response_message = response.choices[0].message.content
    return obj_name, json.loads(strip_code_tags(response_message))

async def query_model(model_id, obj_name, message, images=None):
    use_ollama, providers = MODELS[model_id]
    if use_ollama:
        return await query_model_ollama(model_id, obj_name, message, images=images)
    else:
        return await query_model_openai(model_id, obj_name, message, images=images, providers=providers)

async def query_models(obj_name, message, images=None):
    tasks = []
    model_ids = []
    for use_ollama, model_id in MODELS:
        model_ids.append(model_id)
        if use_ollama:
            tasks.append(query_model_ollama(model_id, message, images))
        else:
            tasks.append(query_model_openai(model_id, message, images))
    results = await asyncio.gather(*tasks)
    return obj_name, {model_id: result[1] for model_id, result in zip(model_ids, results)}

In [ ]:
import traceback
import numpy as np
import tqdm.notebook as tqdm
import pathlib
import asyncio

RESULT_DIR = pathlib.Path(r"D:/ig_pipeline/metadata/vlm_diced_size_predictions/")

async def run_pass(model_id):
    model_suffix = model_id.split("/")[-1]
    result_path = RESULT_DIR / f"{model_suffix}.json"

    results = {}
    if result_path.exists():
        with open(result_path, "r") as f:
            results = json.load(f)

    gpt_missing = [c for c in particle_size_option.keys() if c not in results or results[c] is None]
    print(len(gpt_missing))
    gpt_missing = gpt_missing

    async_futures = []
    for ps_name in gpt_missing:
        message = ""
        ps = ParticleSystem.get(ps_name)
        diced_synset = ps.synset
        assert diced_synset is not None

        # Find the non-diced synset
        synsets = [s for s in diced_synset.derivative_ancestors if not s.is_derivative]
        if not synsets:
            print(f"No non-diced synset found for {ps_name}.")
        synset = synsets[0] if synsets else None

        if synset:
            if synset.definition:
                message += f'"{ps_name}": the result of dicing "{synset.name}", defined as "{synset.definition}"\n'
            else:
                parent_synset = synset.parents[0]
                assert parent_synset.definition
                message += f'"{ps_name}": the result of dicing "{synset.name}", which is a child of {parent_synset.name}, defined as "{parent_synset.definition}"\n'
        else:
            message += f'"{ps_name}": the result of dicing "{ps_name}"\n'

        message += "Human annotator's guess on fine/coarse: " + particle_size_option[ps_name] + "\n"
            
        async_futures.append(query_model(model_id, ps_name, message))

    for result in tqdm.tqdm(asyncio.as_completed(async_futures), total=len(async_futures)):
        try:
            ps_name, result_dict = await result
            # filtered_results = {k: v for k, v in result_dict.items() if k in missing_names}
            results[ps_name] = result_dict["size"] if "size" in result_dict else None

            # Save after every result
            with open(result_path, "w") as f:
                json.dump(results, f)
        except:
            print("Error in system", ps_name)
            traceback.print_exc()

In [ ]:
for model in MODELS.keys():
    print("Running pass for model", model)
    await run_pass(model)

In [ ]:
from collections import defaultdict


results = defaultdict(dict)
for model in MODELS:
    model_suffix = model.split("/")[-1]
    result_path = RESULT_DIR / f"{model_suffix}.json"
    if not result_path.exists():
        continue
    with open(result_path, "r") as f:
        for cat, mass in json.load(f).items():
            results[cat][model] = mass

print(len(results))
# print("Missing categories", len([c for c in Category.all_objects() if c.name not in results]))
# cat_names = {c.name for c in Category.all_objects()}
# print("Extra categories", len([x for x in results.keys() if x not in cat_names]))

In [ ]:
# Check the max/min ratio for each category
ratios = {}
median_sizes = {}
for cat, masses in results.items():
    assert len(masses) == len(MODELS)
    min_mass = min([m for m in masses.values() if m is not None])
    max_mass = max([m for m in masses.values() if m is not None])
    ratio = max_mass / min_mass
    ratios[cat] = ratio
    median_sizes[cat] = np.clip(np.median(list(masses.values())), 0.5, 3.)
ratios = sorted(ratios.items(), key=lambda x: x[1])
print(ratios[:10], ratios[-10:])

In [ ]:
# Show the smallest and largest median sizes
smallest = sorted(median_sizes.items(), key=lambda x: x[1])[:10]
largest = sorted(median_sizes.items(), key=lambda x: x[1], reverse=True)[:10]

print("Smallest systems:")
for cat, masses in smallest:
    print(cat, masses)

print("Largest systems:")
for cat, masses in largest:
    print(cat, masses)

In [ ]:
# How many are smaller than 1cm?
smallest_1cm = [cat for cat, size in median_sizes.items() if size < 1.0]
print("Number of systems with median size < 1cm:", len(smallest_1cm))
print("Examples:", smallest_1cm[:10])

In [ ]:
# Save the results
with open(r"D:/ig_pipeline/metadata/diced_particle_systems_colors.json") as f:
    saved_data = json.load(f)
for particle, infos in saved_data.items():
    infos["llm_size_cm"] = median_sizes[particle]
with open(r"D:/ig_pipeline/metadata/diced_particle_systems_colors.json", "w") as f:
    json.dump(saved_data, f, indent=4)